In [1]:
import pandas as pd
import numpy as np


In [2]:
reg = pd.read_csv('feb16-mar15_data/beneficiary/ai_registration-20210216-20210315.csv', sep='\t')

In [3]:
reg

,user_id,phone_no,lmp,enroll_gest_age,project_id,manager_id,call_slots,enroll_delivery_status,delivery,language,...,ppc_otherComplications,name_of_medication_any,planned_place_of_delivery,registered_where,registered_pregnancy,place_of_delivery,type_of_delivery,date_registration_hospital,term_of_delivery,medication_after_delivery
0,2385610,934332f7f2dac0b09edc5572fcbcded8,2020-11-14,13,3,0,3,0,0,2,...,\N,NaN,Government Hospital,Government Hospital,Yes,\N,\N,2021-02-15,\N,NaN
1,2385611,d154839aef700aeadd4aeea972ec739b,2020-11-05,14,3,0,3,0,0,2,...,\N,NaN,Government Hospital,Government Hospital,Yes,\N,\N,2021-02-15,\N,NaN
2,2385612,1a1893861b32ea57cd86f2c268329995,2020-12-20,8,3,0,6,0,0,3,...,\N,NaN,Government Hospital,Government Hospital,Yes,\N,\N,2021-02-15,\N,NaN
3,2385613,608ff805950471b63b183054b60bf3df,2020-11-25,11,3,0,3,0,0,3,...,\N,NaN,Government Hospital,Government Hospital,Yes,\N,\N,2021-02-15,\N,NaN
4,2385614,d1fd584498c799e3fbda7d9e38f543bd,2020-07-09,31,3,0,3,0,0,2,...,\N,NaN,Government Hospital,Government Hospital,Yes,\N,\N,2021-02-15,\N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23522,2409132,9eb2c55138cc3198edef39e557e51a09,2021-01-14,8,3,0,1,0,0,3,...,\N,NaN,Government Hospital,Government Hospital,Yes,\N,\N,2021-03-10,\N,NaN
23523,2409133,70ae26e929a8625fc32db522954814be,2020-06-12,39,3,0,3,0,1,2,...,\N,NaN,Government Hospital,Government Hospital,Yes,\N,\N,2020-09-20,\N,NaN
23524,2409134,4b16fb66d8899412f57328a720cba0a6,2020-09-11,26,3,0,3,0,0,2,...,\N,NaN,Government Hospital,Government Hospital,Yes,\N,\N,2021-03-10,\N,NaN
23525,2409135,ee32899ab102835a9a47ca2f43753089,2020-10-16,21,3,0,4,0,0,2,...,\N,NaN,\N,Private Hospital,\N,\N,\N,2021-01-16,\N,NaN


In [4]:
ppc_cols = [col for col in reg.columns if col.startswith('ppc')]
reg[ppc_cols] = reg[ppc_cols].replace('\\N', np.nan)
reg[ppc_cols] = (~reg[ppc_cols].isna()).astype(int)

In [5]:
pii = ['data_entry_officer', 'name_of_project_officer', 'name_of_sakhi',
       'alternate_no', 'unique_sub_id', 'entry_madeby', 'phone_no']
date_cols = ['date_registration_hospital', 'updated_status_on', 'updated_time', 'entry_date',
             "registration_date", "delivery_date", 'lmp']

In [6]:
reg = reg.replace('\\N', np.nan)

In [7]:
summary_df = pd.DataFrame(columns=['feat_name', 'type', 'min', 'max', 'mean', 'n_unique_vals', 'proportion_values_missing', '5_sample_vals'])
for col in reg.columns:
    if reg.dtypes[col]=='object':
        row_dict = {'feat_name':col, 'type':'categorical',
                           'n_unique_vals': reg[col].nunique(),
                          'proportion_values_missing': reg[col].isna().mean(),
                          '5_sample_vals': ', '.join(reg[col].unique()[:5].astype(str))}
        if col in pii:
            row_dict['5_sample_vals'] = 'hidden'
        if col in date_cols:
            row_dict['type'] = 'date'
            row_dict['5_sample_vals'] = 'hidden'
    else:
        row_dict = {'feat_name':col, 'type':'numerical',
                           'min': reg[col].min(), 'max': reg[col].max(),
                           'mean': reg[col].mean(),
                           'proportion_values_missing': reg[col].isna().mean()}
    summary_df = summary_df.append(row_dict, ignore_index=True)

In [8]:
pd.options.display.max_rows = 900
pd.set_option('display.float_format', lambda x: '%.3f' % x)
summary_df

,feat_name,type,min,max,mean,n_unique_vals,proportion_values_missing,5_sample_vals
0,user_id,numerical,2385610,2409136,2397373.000,nan,0.000,NaN
1,phone_no,categorical,NaN,NaN,nan,23527.000,0.000,hidden
2,lmp,date,NaN,NaN,nan,453.000,0.000,hidden
3,enroll_gest_age,numerical,0,40,20.382,nan,0.000,NaN
4,project_id,numerical,2,26,6.061,nan,0.000,NaN
5,manager_id,numerical,0,0,0.000,nan,0.000,NaN
6,call_slots,numerical,1,6,3.556,nan,0.000,NaN
7,enroll_delivery_status,numerical,0,1,0.146,nan,0.000,NaN
8,delivery,numerical,0,2,0.259,nan,0.000,NaN
9,language,numerical,2,5,2.500,nan,0.000,NaN


In [37]:
summary_df.to_csv('outputs/registration_data_summary.csv', index=False)

In [43]:
reg['place_of_delivery'].unique()

array(['\\N', 'Government Hospital', 'Private Hospital', 'Home'],
      dtype=object)

In [14]:
(reg['enroll_phone_owner'] == reg[ 'phone_owner']).sum()

23527

In [21]:
reg['project_id'].unique()

array([ 3, 13,  2, 19,  9, 18, 26])